# 📓 OCSVM - 2F - Validación por Repetición
# Basado en nº de outliers

**Proyecto**: Detección de posibles fuentes de materia oscura usando ML en datos Fermi-LAT  
**Autor**: Marta Canino Romero  
**Fecha**: mayo 2025

---

## ✨ Descripción:

Este notebook implementa un modelo **One-Class SVM** entrenado con datos de fuentes astrofísicas conocidas (ASTRO) usando las siguientes características:

- E_peak
- beta

**A diferencia de otros notebooks, aquí el proceso se repite múltiples veces (con diferentes semillas aleatorias en los splits) para comprobar la robustez y variabilidad del modelo.** Cada iteración realiza:

1. División de datos (train/val/test)
2. Escalado
3. Búsqueda de hiperparámetros (grid search sobre `nu` y `gamma`)
4. Evaluación sobre validación y prueba
5. Registro de métricas clave (f1-score, número de outliers, matriz de confusión)

Los resultados de cada iteración se almacenan y analizan para estimar la estabilidad del modelo.

---

## 📌 Objetivos específicos:

- Evaluar la variabilidad del modelo OCSVM tras `N` repeticiones
- Analizar la dispersión del mejor f1-score entre repeticiones
- Comparar número de anomalías detectadas en cada iteración
- Obtener métricas medias y desviaciones estándar

---

## 🗂️ Entrada de datos:

- `../../data/processed/XY_bal_log_Rel/astro/XY_bal_log_Rel_astro.txt`

## 💾 Salida esperada:

- Tabla resumen de métricas por iteración
- Mejor combinación de hiperparámetros promedio
- Exportación de anomalías más recurrentes


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score, classification_report, confusion_matrix


In [ ]:
# Load dataset
# data_path = "../../data/processed/XY_bal_log_Rel/astro/XY_bal_log_Rel_astro.txt"
data_path = "../../data/processed/XY_bal_log_Rel/astro/astro_df.txt"
df_astro = pd.read_csv(data_path, sep='\s+')

df_astro = df_astro.rename(columns={"0,1=astro,DM": "class"})
print(df_astro.columns)

df_astro.head()

In [ ]:
# --- Selección de características ---
features = ['Log(E_peak)', 'Log(beta)']
target = 'astro_DM'

print(f"Features seleccionadas: {features}")
print(f"Columna objetivo: {target}")

# --- Comprobamos valores nulos ---
print("\n Valores faltantes por columna:")
print(df_astro[features + [target]].isnull().sum())

print("\n Muestra del dataset:")
display(df_astro[features + [target]].head())

In [ ]:
# Comprobamos la distribución de los datos astro antes de escalar
plt.figure(figsize=(8, 6))

sns.scatterplot(
    data=df_astro,
    x="Log(E_peak)",
    y="Log(beta)",
    color="cornflowerblue",  # "skyblue" "cornflowerblue"
    edgecolor='k',
    alpha=0.7,
    s=40
)

plt.title("ASTRO Data: Log(E_peak) vs Log(Beta)")
plt.xlabel("Log(E_peak)")
plt.ylabel("Log(Beta)")
plt.grid(True)
plt.show()

In [ ]:

# 💥 Número de repeticiones
n_iterations = 5

# Guardar resultados de todas las iteraciones
all_iterations_results = []
best_models = []  # opcional: guardar modelo por iteración

print("🔍 Buscando hiperparámetros que minimicen outliers en ASTRO (validación)...")
for i in range(n_iterations):
    print(f"\n🔄 Iteración {i+1}/{n_iterations}")

    # =============== 1️⃣ Split dinámico por iteración ===============
    X = df_astro[features].values
    y = df_astro[target].values

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.4, stratify=y, random_state=42 + i
    )

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=99 + i
    )

    # =============== 2️⃣ Escalado ===============
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # =============== 3️⃣ Grid search en esta iteración ===============
    nu_values = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05]
    # gamma_values = ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10]
    gamma_values = [0.1] 

    best_score = 0.0
    iteration_results = []
    best_outliers = np.inf
    best_model_iter = None
    best_params_iter = {}

    for nu in nu_values:
        for gamma in gamma_values:
            model = OneClassSVM(kernel='rbf', nu=nu, gamma=gamma)
            model.fit(X_train_scaled)

            preds = model.predict(X_val_scaled)
            pred_labels = np.where(preds == 1, 0, 1)  # 1→normal, -1→outlier
            n_outliers = np.sum(preds == -1)
            true_labels = y_val.astype(int)

            # f1 = f1_score(true_labels, pred_labels, pos_label=0)

            iteration_results.append({'nu': nu, 'gamma': gamma, 'val_outliers': n_outliers})

            """
            if f1 > best_score:
                best_score = f1
                best_model_iter = model
                best_params_iter = {'nu': nu, 'gamma': gamma}
            """
            if n_outliers < best_outliers:
                best_outliers = n_outliers
                best_model_iter = model
                best_params_iter = {'nu': nu, 'gamma': gamma}

    print(f"✅ Mejor iteración {i+1}: nu={best_params_iter['nu']}, gamma={best_params_iter['gamma']}, Outliers (val set): {best_outliers} de {len(X_val_scaled)} muestras")

    # Guardar resultados de esta iteración
    all_iterations_results.extend(
        [dict(iter=i+1, **res) for res in iteration_results]
    )

    best_models.append(best_model_iter)  # opcional

# onvertimos a DataFrame global
df_all_results = pd.DataFrame(all_iterations_results)

# Mostrar el top global
display(df_all_results.sort_values(by='val_outliers', ascending=True))


In [ ]:
from sklearn.utils.multiclass import unique_labels

# Paso 1: identificar hiperparámetros globales
best_global = df_all_results.sort_values(by='val_outliers', ascending=True).iloc[0]
print("Mejor combinación global:")
print(best_global)

# Paso 2: reentrenar con X_train + X_val
X_final_train = np.vstack([X_train, X_val])
y_final_train = np.concatenate([y_train, y_val])

scaler_final = StandardScaler()
X_final_train_scaled = scaler_final.fit_transform(X_final_train)
X_test_scaled = scaler_final.transform(X_test)

# Paso 3: entrenar modelo final
final_model = OneClassSVM(kernel='rbf', nu=best_global['nu'], gamma=best_global['gamma'])
final_model.fit(X_final_train_scaled)

# Paso 4: evaluar en test
test_preds = final_model.predict(X_test_scaled)
test_labels = np.where(test_preds == 1, 0, 1)

# Paso 5: métricas
print("\nEvaluación en el conjunto de test:")
print(f"Outliers (test set): {np.sum(test_preds == -1)} de {len(X_test_scaled)} muestras")
# print(f"F1 Score: {f1_score(y_test, test_labels, pos_label=0):.4f}")
print("\nMatriz de confusión:")
cm = confusion_matrix(y_test, test_labels)
print(cm)
print("\nReporte de clasificación:")
print(classification_report(y_test, test_labels, target_names=unique_labels(y_test, test_labels).astype(str)))

In [ ]:
# Create meshgrid (with correct feature order)
xx, yy = np.meshgrid(
    np.linspace(X_test_scaled[:, 0].min() - 0.5, X_test_scaled[:, 0].max() + 0.5, 300),  # E_peak
    np.linspace(X_test_scaled[:, 1].min() - 0.5, X_test_scaled[:, 1].max() + 0.5, 300)   # beta
)

grid = np.c_[xx.ravel(), yy.ravel()]

Z = final_model.decision_function(grid)
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu_r, alpha=0.8)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='crimson', linestyles='--')

# Plot training and test data
plt.scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], c='darkblue', edgecolors='k', s=40, label='Test')
plt.scatter(X_final_train_scaled[:, 0], X_final_train_scaled[:, 1], c='skyblue', edgecolors='k', s=40, label='Train')

plt.xlabel("E_peak (scaled)")
plt.ylabel("beta (scaled)")
plt.title("Best One-Class SVM Decision Boundary")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Predict on UNIDs

In [ ]:
# unids_path = "../../data/raw/unids_3F_beta_err_names.txt"
unids_path = "../../data/processed/unids_log/unids_log.txt"
df_unids = pd.read_csv(unids_path, sep='\s+')
df_unids.head()

In [ ]:
# Extraer y escalar
X_unids_log = df_unids[["Log(E_peak)", "Log(beta)"]].values
X_unids_scaled = scaler.transform(X_unids_log)

In [ ]:
# --- Create subplots ---
fig, axes = plt.subplots(1, 2, figsize=(20, 6), sharey=True)

# --- Plot 1: Raw UNIDS data ---
sns.scatterplot(
    data=df_unids,
    x="Log(E_peak)",
    y="Log(beta)",
    color="gold",
    edgecolor='k',
    alpha=0.7,
    s=40,
    ax=axes[0]
)
axes[0].set_title("UNIDS Data: E_peak vs Beta")
axes[0].set_xlabel("Log(E_peak)")
axes[0].set_ylabel("Log(beta)")
axes[0].grid(True)

# --- Plot 3: Scaled UNIDS ---
sns.scatterplot(
    x=X_unids_scaled[:, 0],
    y=X_unids_scaled[:, 1],
    color="gold",
    edgecolor='k',
    alpha=0.7,
    s=40,
    ax=axes[1]
)
axes[1].set_title("Scaled UNIDS: E_peak vs Beta")
axes[1].set_xlabel("Log(E_peak) (scaled)")
axes[1].set_ylabel("Log(beta) (scaled)")
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Scatter de unids escalados vs datos de entrenamiento escalados
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x=X_final_train_scaled[:, 0],
    y=X_final_train_scaled[:, 1],
    color="skyblue",  # "skyblue" "cornflowerblue"
    edgecolor='k',
    alpha=0.7,
    s=40,
    label='Astro'
)
sns.scatterplot(
    x=X_unids_scaled[:, 0],
    y=X_unids_scaled[:, 1],
    color="gold",  # "skyblue" "cornflowerblue"
    edgecolor='k',
    alpha=0.7,
    s=40,
    label='Unids'
)
plt.title("Scaled UNIDS Data vs Train Data: E_peak vs Beta")
plt.xlabel("Log(E_peak) (scaled)")
plt.ylabel("Log(beta) (scaled)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Predecir con el modelo final
unids_preds = final_model.predict(X_unids_scaled)
unids_labels = np.where(unids_preds == 1, 0, 1)  # 1→normal, -1→outlier

n_outliers = np.sum(unids_preds == -1)
n_normals = np.sum(unids_preds == 1)

print(f"Predicted ASTRO-like: {n_normals}")
print(f"Predicted not ASTRO-like (anomalies): {n_outliers}")

# Añadir etiquetas al DataFrame
df_unids['pred_labels'] = unids_labels

# Filtrar los datos no clasificados
df_unids['pred_labels'] = df_unids['pred_labels'].astype(int)
df_unids['pred_labels'] = df_unids['pred_labels'].replace({0: 'Normal', 1: 'Anomalía'})


In [ ]:
df_unids.head()

In [ ]:
inliers = X_unids_scaled[unids_preds == 1]
outliers = X_unids_scaled[unids_preds == -1]

plt.figure(figsize=(8, 6))

plt.scatter(inliers[:, 0], inliers[:, 1], c='gold', edgecolors='k', label='Inlier (likely astro)', alpha=0.6)
plt.scatter(outliers[:, 0], outliers[:, 1], c='red', edgecolors='k', label='Outlier (potentially new)', alpha=0.6)

plt.xlabel("Log(E_peak) (scaled)")
plt.ylabel("Log(beta) (scaled)")

plt.title("UNIDs: Inlier vs Outlier (One-Class SVM)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Create meshgrid (with correct feature order: [beta, E_peak])
xx, yy = np.meshgrid(
    np.linspace(X_train_scaled[:, 0].min() - 0.5, X_train_scaled[:, 0].max() + 0.5, 300),  # E_peak
    np.linspace(X_train_scaled[:, 1].min() - 0.5, X_train_scaled[:, 1].max() + 0.5, 300)   # beta
)

grid = np.c_[xx.ravel(), yy.ravel()]  # correcto orden

Z = final_model.decision_function(grid)
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu_r, alpha=0.8)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='crimson', linestyles='--')

preds_unids = final_model.predict(X_unids_scaled)
inliers = X_unids_scaled[preds_unids == 1]
outliers = X_unids_scaled[preds_unids == -1]

plt.scatter(inliers[:, 0], inliers[:, 1], c='gold', edgecolors='k', s=40, label='UNIDs Inlier')
plt.scatter(outliers[:, 0], outliers[:, 1], c='red', edgecolors='k', s=40, label='UNIDs Outlier')

# Highlight most anomalous UNIDs using their lowest decision score:
decision_scores = final_model.decision_function(X_unids_scaled)
# Optional: mark top 5 most anomalous
top_anomalies = X_unids_scaled[np.argsort(decision_scores)[:5]]
plt.scatter(top_anomalies[:, 1], top_anomalies[:, 0], c='yellow', edgecolors='black', s=80, label='Top Outliers', marker='*')


plt.xlabel("Log(E_peak) (scaled)")
plt.ylabel("Log(beta) (scaled)")
plt.title("Predictions on UNIDs with One-Class SVM Decision Boundary")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Anomaly Scoring - UNIDs

In [ ]:
# Paso 1: Evaluar cada muestra no identificada con el modelo entrenado
# decision_function devuelve un valor continuo: cuanto más alto, más normal (positivo); cuanto más bajo, más anómalo (negativo)
decision_scores = final_model.decision_function(X_unids_scaled)  # X_unids_scaled = muestras no etiquetadas, ya escaladas

# Paso 2: Predecir si cada punto es inlier (1) o outlier (-1)
unids_preds = final_model.predict(X_unids_scaled)  # 1 = normal, -1 = anomalía

# Paso 3: Agregar los resultados al DataFrame original
df_unids["svm_score"] = decision_scores       # Puntaje bruto del modelo (positivo = normal)
df_unids["prediction"] = unids_preds          # Clasificación binaria: inlier o outlier

# Paso 4: Invertimos el score para que valores más altos signifiquen más anomalía
# Esto es útil para poder escalar la puntuación y ordenar más intuitivamente
anom_scores = -decision_scores  # Ahora, valores grandes = más anómalos

# Paso 5: Escalamos los scores de anomalía al rango [0, 100] para facilitar su interpretación
anom_percent = MinMaxScaler(feature_range=(0, 100)).fit_transform(anom_scores.reshape(-1, 1)).flatten()

# Guardamos el puntaje invertido y su porcentaje normalizado en el DataFrame
df_unids["Anomaly_Score"] = anom_scores
df_unids["Anomaly_Rank(%)"] = anom_percent  # 100 = más anómalo, 0 = más normal

# Paso 6: Filtramos solo los puntos predichos como anómalos y los ordenamos por su score más alto
top_anomalies = df_unids[df_unids["prediction"] == -1] \
                    .sort_values(by="Anomaly_Rank(%)", ascending=False) \
                    .head(10)

# Guardamos los índices (puede ser útil si queremos recuperar sus posiciones originales)
most_anomalous_idx = top_anomalies.index

# Paso 7: Guardamos los 10 más anómalos en un archivo
# top_anomalies.to_csv("../../data/processed/unids_most_anomalous.txt", sep="\t", index=False)

# Paso 8: Mostramos en pantalla un resumen de las anomalías detectadas
print("Top Most Anomalous UNID Sources (4F One-Class SVM):")
display(top_anomalies[['Log(E_peak)', 'Log(beta)', 'number', 'svm_score', 'Anomaly_Score', 'Anomaly_Rank(%)']])

In [ ]:
# Sort anomaly scores and grab top N labels
N = 10
sorted_idx = np.argsort(-anom_percent)  # high anomaly % = more anomalous
top_N_idx = sorted_idx[:N]

top_labels = df_unids.iloc[top_N_idx]['number'].astype(str).values
top_scores = anom_percent[top_N_idx]

plt.figure(figsize=(12, 6))
plt.hist(anom_percent, bins=50, color='blue', edgecolor='k', alpha=0.7)
plt.title("Anomaly Percentage Distribution (UNID Sources)")
plt.xlabel("Anomaly % (higher = more anomalous)")
plt.ylabel("Number of Sources")

for i in range(N):
    x = top_scores[i]
    label = top_labels[i]
    plt.axvline(x, color='crimson', linestyle='--', alpha=0.8)
    plt.text(x + 0.5, 3 + (i % 2) * 2, f"ID {label}", rotation=90, color='crimson', ha='left', fontsize=9)

plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(9, 6))

# Inliers and outliers from scaled data
inliers = X_unids_scaled[unids_preds == 1]
outliers = X_unids_scaled[unids_preds == -1]

# Plot inliers (gold)
plt.scatter(inliers[:, 0], inliers[:, 1], c='gold', edgecolors='k', label='Inlier (astro-like)', alpha=0.6)

# Plot outliers (red)
plt.scatter(outliers[:, 0], outliers[:, 1], c='red', edgecolors='k', label='Outlier (anomalous)', alpha=0.6)

# Annotate top 10 anomalies by ID
for idx in most_anomalous_idx:
    x = X_unids_scaled[idx, 0]  # E_peak (scaled)
    y = X_unids_scaled[idx, 1]  # beta (scaled)
    label = int(df_unids.loc[idx, 'number'])
    plt.text(x + 0.1, y, str(label), color='black', fontsize=9)

# Axis labels and styling
plt.xlabel("E_peak (scaled)")
plt.ylabel("beta (scaled)")
plt.title("UNID Sources (2F) – Inliers vs Anomalies with ID Labels")
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig("../../outputs/figures/scaled/2F_UNIDs_OneClassSVM_2D_scaled.png", dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Original/log-transformed values for 3 relevant features
x_vals = df_unids['Log(E_peak)'].values
y_vals = df_unids['Log(beta)'].values
z_vals = df_unids['Log(sigma_det)'].values  # or 'beta_Rel' for 3F.2

# Use model predictions for coloring
inlier_idx = df_unids['prediction'] == 1
outlier_idx = df_unids['prediction'] == -1

# Inliers
ax.scatter(
    x_vals[inlier_idx], y_vals[inlier_idx], z_vals[inlier_idx],
    c='#1f77b4', edgecolor='k', s=30, label='Inlier (astro-like)', alpha=0.5
)

# Outliers
ax.scatter(
    x_vals[outlier_idx], y_vals[outlier_idx], z_vals[outlier_idx],
    c='#d62728', marker='^', edgecolor='k', s=50, label='Outlier (anomaly)', alpha=0.9
)

# Annotate top anomalies
top_anomalies = df_unids[df_unids['prediction'] == -1].sort_values('Anomaly_Rank(%)', ascending=False).head(10)
for idx in top_anomalies.index:
    ax.scatter(
        x_vals[idx], y_vals[idx], z_vals[idx],
        facecolors='none', edgecolors='black', linewidths=2, s=100
    )
    ax.text(
        x_vals[idx], y_vals[idx], z_vals[idx] + 0.05,
        str(int(df_unids.loc[idx, 'number'])),
        color='black', fontsize=9
    )

ax.set_xlabel('E_peak')
ax.set_ylabel('beta')
ax.set_zlabel('sigma_det')  # or 'beta_Rel'
ax.set_title("2F UNID Sources – Anomalies in Original Feature Space")
ax.legend(loc='upper left')

ax.tick_params(colors='#333333')
ax.xaxis.label.set_color('#333333')
ax.yaxis.label.set_color('#333333')
ax.zaxis.label.set_color('#333333')
ax.title.set_color('#111111')
ax.grid(color='#aaaaaa', linestyle='--', alpha=0.3)

plt.savefig("../../outputs/figures/2F_UNIDs_OneClassSVM_og.png", dpi=300, bbox_inches='tight')

plt.tight_layout()
plt.show()

# Comparar con UNIDs ANN

In [ ]:
from scipy.stats import spearmanr, pearsonr
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
# Load and process supervised model predictions
print("Loading supervised model predictions...")

# Load predictions data
unids_DM_std_proba_repeated_kfold = np.genfromtxt('../../ANN_original/unids_DM_std_proba_check_repeated_kfold_2F_21.txt', dtype='str') 
unids_DM_std_proba_data_repeated_kfold = np.asarray(unids_DM_std_proba_repeated_kfold[1::], dtype=float)

print(f"Raw predictions shape: {unids_DM_std_proba_data_repeated_kfold.shape}")

# Get dimensions
# Load unID source data
unids_3F = np.genfromtxt('../../ANN_original/unids_3F_beta_err_names.txt',dtype='str') 
unids_3F_data = np.asarray(unids_3F[1::,:],dtype=float)
unids_log=np.log10(unids_3F_data[:,[0,1]])
N_unids = unids_log.shape[0]
# N_sample = unids_DM_std_proba_data_repeated_kfold.shape[1] - 1  # CV folds
N_sample = 10

print(f"Number of sources: {N_unids}")
print(f"Number of CV folds: {N_sample}")

In [ ]:
# Extract indices and reshape probabilities
unids_number = unids_DM_std_proba_data_repeated_kfold[0:N_unids, 0]
prob_values = unids_DM_std_proba_data_repeated_kfold[:, 1].reshape(N_unids, N_sample)

# Create probability matrix
unids_DM_std_proba_N_sample_repeated_kfold = np.zeros((N_unids, N_sample + 1))
unids_DM_std_proba_N_sample_repeated_kfold[:, 0] = unids_number[:].astype(int)
unids_DM_std_proba_N_sample_repeated_kfold[:, 1:(N_sample + 1)] = prob_values

print(f"Probability matrix shape: {unids_DM_std_proba_N_sample_repeated_kfold.shape}")
print("Sample of probability matrix:")
print(unids_DM_std_proba_N_sample_repeated_kfold[:3, :])

In [ ]:
# Extract probability data (excluding index column)
prob_data = unids_DM_std_proba_N_sample_repeated_kfold[:, 1:(N_sample + 1)]

# Calculate statistics across CV folds
unids_mean = np.mean(prob_data, axis=1)
unids_std = np.std(prob_data, axis=1)
unids_median = np.median(prob_data, axis=1)
unids_min = np.min(prob_data, axis=1)
unids_max = np.max(prob_data, axis=1)
unids_q25 = np.percentile(prob_data, 25, axis=1)
unids_q75 = np.percentile(prob_data, 75, axis=1)

print("Supervised model statistics calculated:")
print(f"  Mean probability range: {unids_mean.min():.4f} - {unids_mean.max():.4f}")
print(f"  Standard deviation range: {unids_std.min():.4f} - {unids_std.max():.4f}")

In [ ]:
# Create supervised model DataFrame
supervised_df = pd.DataFrame({
    'source_id': unids_number.astype(int),
    'supervised_mean_prob': unids_mean,
    'supervised_std_prob': unids_std,
    'supervised_median_prob': unids_median,
    'supervised_min_prob': unids_min,
    'supervised_max_prob': unids_max,
    'supervised_q25': unids_q25,
    'supervised_q75': unids_q75,
    'supervised_cv_range': unids_max - unids_min,
    'E_peak': unids_3F_data[:, 0],
    'Beta': unids_3F_data[:, 1],
    'log_E_peak': unids_log[:, 0],
    'log_Beta': unids_log[:, 1]
})

print(f"Supervised DataFrame created with {len(supervised_df)} sources")
print("\nSupervised model top 10 candidates:")
display(supervised_df.nlargest(10, 'supervised_mean_prob')[['source_id', 'supervised_mean_prob', 'supervised_std_prob', 'E_peak', 'Beta']].round(4))

In [ ]:
# Merge with OCSVM results
print("Merging supervised and OCSVM results...")

# Check available columns in OCSVM DataFrame
print("OCSVM DataFrame columns:", df_unids.columns.tolist())

# Merge datasets - adjust merge column as needed
if 'number' in df_unids.columns:
    combined_df = pd.merge(supervised_df, df_unids, left_on='source_id', right_on='number', how='inner')
elif 'source_id' in df_unids.columns:
    combined_df = pd.merge(supervised_df, df_unids, on='source_id', how='inner')
else:
    # If no common key, merge by index (assuming same order)
    print("Warning: No common identifier found, merging by index")
    combined_df = pd.concat([supervised_df.reset_index(drop=True), 
                           df_unids.reset_index(drop=True)], axis=1)

print(f"Combined dataset shape: {combined_df.shape}")
print(f"Combined dataset columns: {combined_df.columns.tolist()}")


In [ ]:
# Calculate model correlations
print("Calculating model correlations...")

# Check if required columns exist
required_cols = ['supervised_mean_prob', 'Anomaly_Rank(%)']
missing_cols = [col for col in required_cols if col not in combined_df.columns]

if missing_cols:
    print(f"Error: Missing columns {missing_cols}")
    print("Available columns:", combined_df.columns.tolist())
else:
    # Calculate correlations
    corr_pearson, p_pearson = pearsonr(combined_df['supervised_mean_prob'], 
                                      combined_df['Anomaly_Rank(%)'])
    corr_spearman, p_spearman = spearmanr(combined_df['supervised_mean_prob'], 
                                         combined_df['Anomaly_Rank(%)'])
    
    print(f"\\nMODEL CORRELATION RESULTS:")
    print(f"  Pearson correlation: {corr_pearson:.4f} (p-value: {p_pearson:.4e})")
    print(f"  Spearman correlation: {corr_spearman:.4f} (p-value: {p_spearman:.4e})")
    
    # Interpretation
    if abs(corr_spearman) > 0.7:
        print("  → Strong correlation between models")
    elif abs(corr_spearman) > 0.4:
        print("  → Moderate correlation between models")
    else:
        print("  → Weak correlation between models")

In [ ]:
# Analyze top candidates overlap
top_n = 20
print(f"\\nAnalyzing top {top_n} candidates from each model...")

# Get top candidates from each model
top_supervised = combined_df.nlargest(top_n, 'supervised_mean_prob')
top_anomalous = combined_df.nlargest(top_n, 'Anomaly_Rank(%)')

# Find overlap
supervised_ids = set(top_supervised.index)
anomaly_ids = set(top_anomalous.index)
overlap_ids = supervised_ids.intersection(anomaly_ids)

# Calculate overlap metrics
total_overlap = len(overlap_ids)
overlap_percentage = total_overlap / top_n * 100
jaccard_index = total_overlap / len(supervised_ids.union(anomaly_ids))

print(f"\\nTOP {top_n} CANDIDATES OVERLAP:")
print(f"  Sources in both top {top_n}: {total_overlap}")
print(f"  Overlap percentage: {overlap_percentage:.1f}%")
print(f"  Jaccard similarity index: {jaccard_index:.4f}")

# Show overlapping candidates
if total_overlap > 0:
    overlap_sources = combined_df.loc[list(overlap_ids)]
    print(f"\\nCONSENSUS CANDIDATES (in both top {top_n}):")
    display_cols = ['source_id', 'supervised_mean_prob', 'Anomaly_Rank(%)', 'F_peak', 'Beta']
    available_cols = [col for col in display_cols if col in overlap_sources.columns]
    display(overlap_sources[available_cols].round(4))


In [ ]:
# Create consensus ranking
print("Creating consensus ranking...")

# Normalize scores to [0, 1] range
supervised_norm = MinMaxScaler().fit_transform(
    combined_df['supervised_mean_prob'].values.reshape(-1, 1)
).flatten()

ocsvm_norm = combined_df['Anomaly_Rank(%)'] / 100

# Create weighted consensus score (adjust weights as needed)
weights = {'supervised': 0.6, 'ocsvm': 0.4}
consensus_score = (weights['supervised'] * supervised_norm + 
                  weights['ocsvm'] * ocsvm_norm)

# Add consensus columns to DataFrame
combined_df['supervised_norm'] = supervised_norm
combined_df['ocsvm_norm'] = ocsvm_norm
combined_df['consensus_score'] = consensus_score
combined_df['consensus_rank'] = combined_df['consensus_score'].rank(ascending=False, method='dense')

print(f"Consensus ranking created with weights: {weights}")
print(f"Consensus score range: {consensus_score.min():.4f} - {consensus_score.max():.4f}")


In [ ]:
# Display top consensus candidates
print("\\nTOP 15 CONSENSUS DARK MATTER CANDIDATES:")
print("="*80)

# Sort by consensus score and display
consensus_top = combined_df.sort_values('consensus_score', ascending=False)

display_cols = ['source_id', 'consensus_rank', 'consensus_score', 
               'supervised_mean_prob', 'supervised_std_prob', 
               'Anomaly_Rank(%)', 'E_peak', 'Beta']
available_display_cols = [col for col in display_cols if col in consensus_top.columns]

display(consensus_top[available_display_cols].head(15).round(4))


In [ ]:
# Create comparison visualizations
print("Creating comparison visualizations...")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Scatter plot: Supervised prob vs Anomaly rank
axes[0, 0].scatter(combined_df['supervised_mean_prob'], 
                  combined_df['Anomaly_Rank(%)'], 
                  alpha=0.6, s=40, c='blue')
axes[0, 0].set_xlabel('Supervised Model Probability')
axes[0, 0].set_ylabel('OCSVM Anomaly Rank (%)')
axes[0, 0].set_title('Supervised vs OCSVM Scores')
axes[0, 0].grid(True, alpha=0.3)

# Add correlation text
axes[0, 0].text(0.05, 0.95, 
               f"Pearson: {corr_pearson:.3f}\\nSpearman: {corr_spearman:.3f}",
               transform=axes[0, 0].transAxes, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 2. Distribution comparison
axes[0, 1].hist(combined_df['supervised_mean_prob'], bins=30, alpha=0.7, 
               label='Supervised Prob', density=True, color='blue')
axes[0, 1].hist(combined_df['Anomaly_Rank(%)']/100, bins=30, alpha=0.7, 
               label='OCSVM Anomaly Rank', density=True, color='red')
axes[0, 1].set_xlabel('Score')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Score Distributions')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Consensus score distribution
axes[0, 2].hist(combined_df['consensus_score'], bins=30, alpha=0.7, 
               color='green', edgecolor='black')
axes[0, 2].set_xlabel('Consensus Score')
axes[0, 2].set_ylabel('Count')
axes[0, 2].set_title('Consensus Score Distribution')
axes[0, 2].grid(True, alpha=0.3)

# 4. Feature space with supervised model coloring
scatter1 = axes[1, 0].scatter(combined_df['E_peak'], combined_df['Beta'], 
                            c=combined_df['supervised_mean_prob'], 
                            cmap='viridis', alpha=0.7, s=40)
axes[1, 0].set_xlabel('E_peak')
axes[1, 0].set_ylabel('Beta')
axes[1, 0].set_title('Feature Space (Supervised Prob)')
axes[1, 0].set_xscale('log')
axes[1, 0].set_yscale('log')
plt.colorbar(scatter1, ax=axes[1, 0])

# 5. Feature space with OCSVM colorin
scatter2 = axes[1, 1].scatter(combined_df['E_peak'], combined_df['Beta'], 
                            c=combined_df['Anomaly_Rank(%)'], 
                            cmap='plasma', alpha=0.7, s=40)
axes[1, 1].set_xlabel('E_peak')
axes[1, 1].set_ylabel('Beta')
axes[1, 1].set_title('Feature Space (OCSVM Anomaly)')
axes[1, 1].set_xscale('log')
axes[1, 1].set_yscale('log')
plt.colorbar(scatter2, ax=axes[1, 1])

# 6. Feature space with consensus coloring
scatter3 = axes[1, 2].scatter(combined_df['E_peak'], combined_df['Beta'], 
                            c=combined_df['consensus_score'], 
                            cmap='coolwarm', alpha=0.7, s=40)
axes[1, 2].set_xlabel('E_peak')
axes[1, 2].set_ylabel('Beta')
axes[1, 2].set_title('Feature Space (Consensus Score)')
axes[1, 2].set_xscale('log')
axes[1, 2].set_yscale('log')
plt.colorbar(scatter3, ax=axes[1, 2])

plt.tight_layout()
plt.show()


In [ ]:
#  Model agreement analysis
print("\\nAnalyzing model agreement across score ranges...")

# Create score bins for agreement analysis
supervised_bins = pd.cut(combined_df['supervised_mean_prob'], 
                        bins=5, labels=['Low', 'Med-Low', 'Med', 'Med-High', 'High'])
anomaly_bins = pd.cut(combined_df['Anomaly_Rank(%)'], 
                     bins=5, labels=['Low', 'Med-Low', 'Med', 'Med-High', 'High'])

# Create agreement matrix
agreement_matrix = pd.crosstab(supervised_bins, anomaly_bins, margins=True)
print("\\nModel Agreement Matrix:")
print("(Rows: Supervised Model, Columns: OCSVM)")
display(agreement_matrix)

# Plot agreement heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(agreement_matrix.iloc[:-1, :-1], annot=True, fmt='d', 
            cmap='Blues', cbar_kws={'label': 'Number of Sources'})
plt.xlabel('OCSVM Anomaly Level')
plt.ylabel('Supervised Model Probability Level')
plt.title('Model Agreement Heatmap')
plt.show()


In [ ]:
# Summary statistics and final report
print("\\n" + "="*80)
print("DARK MATTER CANDIDATE MODEL COMPARISON SUMMARY")
print("="*80)

print(f"\\nDATASET OVERVIEW:")
print(f"  Total sources analyzed: {len(combined_df)}")
print(f"  Supervised model probability range: {combined_df['supervised_mean_prob'].min():.4f} - {combined_df['supervised_mean_prob'].max():.4f}")
print(f"  OCSVM anomaly rank range: {combined_df['Anomaly_Rank(%)'].min():.1f}% - {combined_df['Anomaly_Rank(%)'].max():.1f}%")
print(f"  Consensus score range: {combined_df['consensus_score'].min():.4f} - {combined_df['consensus_score'].max():.4f}")

print(f"\\nMODEL PERFORMANCE:")
print(f"  Cross-validation folds used: {N_sample}")
print(f"  Mean supervised model uncertainty: {combined_df['supervised_std_prob'].mean():.4f}")
print(f"  Sources with high confidence (>90%): {sum(combined_df['supervised_mean_prob'] > 0.9)}")
print(f"  Sources with low uncertainty (<10%): {sum(combined_df['supervised_std_prob'] < 0.1)}")

print(f"\\nMODEL AGREEMENT:")
print(f"  Pearson correlation: {corr_pearson:.4f}")
print(f"  Spearman correlation: {corr_spearman:.4f}")
print(f"  Top {top_n} overlap: {total_overlap} sources ({overlap_percentage:.1f}%)")
print(f"  Jaccard similarity: {jaccard_index:.4f}")

# Identify most promising candidates
high_consensus = combined_df[combined_df['consensus_score'] > 0.8]
print(f"\\nHIGH-CONFIDENCE CONSENSUS CANDIDATES:")
print(f"  Sources with consensus score >0.8: {len(high_consensus)}")

if len(high_consensus) > 0:
    print("\\nTop 5 highest consensus candidates:")
    top_consensus_cols = ['source_id', 'consensus_score', 'supervised_mean_prob', 'Anomaly_Rank(%)']
    available_consensus_cols = [col for col in top_consensus_cols if col in high_consensus.columns]
    display(high_consensus[available_consensus_cols].head().round(4))
